In [ ]:
!pip install spacy
!pip install rank-bm25
!pip install nmslib
import os
import pandas as pd
import numpy as np
import pickle
import spacy
from tqdm import tqdm
import matplotlib.pyplot as plt
from gensim.models.fasttext import FastText
!pip install rank_bm25 --quiet #install BM25
!pip install --no-binary :all: nmslib #install nmslib
from rank_bm25 import BM25Okapi
import time
import nmslib
#from google.colab import drive

# drive.mount('/content/drive') # if you want to use G Drive
#pd.set_option('display.max_colwidth', -1)
#plt.style.use('fivethirtyeight')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df = pd.read_csv('run_results.csv')
df['text'] = df['Tender_name'] + ' ' + df['Tender_Tender_desc']+ ' ' + df['Tender_Publication_date']+ ' ' + df['Tender_Closing_date']+ ' ' + df['Tender_Contract_value']+ ' ' + df['Tender_Location']
df.shape


(220, 8)

In [ ]:
df.head(1)

,Tender_name,Tender_url,Tender_Tender_desc,Tender_Publication_date,Tender_Closing_date,Tender_Contract_value,Tender_Location,text
0,CA11819 - Hebburn Extra Care Development scheme,https://www.contractsfinder.service.gov.uk/not...,An Open tender is being carried out under the ...,"Publication date 7 February 2023, last edited ...","Closing 24 March 2023, 12pm","Contract value £26,000,000",Contract location NE12 8EG,CA11819 - Hebburn Extra Care Development schem...


In [ ]:
def fix_text(x):
  """This function does nothing but leaving
  here as a placeholder for basic text cleaning"""
  return x

nlp = spacy.load("en_core_web_sm")
tok_text=[] # for our tokenised corpus
text = df.text.str.lower().values
text = [fix_text(str(i)) for i in text]

#Tokenising using SpaCy:
for doc in tqdm(nlp.pipe(text, disable=["tagger", "parser","ner"]), total=len(text), desc="Tokenizing text"):
    tok = [t.text for t in doc if (t.is_ascii and not t.is_punct and not t.is_space)]
    tok_text.append(tok)

Tokenizing text:   0%|          | 0/220 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
Tokenizing text: 100%|██████████| 220/220 [00:01<00:00, 178.84it/s]


In [ ]:
ft_model = FastText(
    sg=1, # use skip-gram: usually gives better results
    size=100, # embedding dimension (default)
    window=10, # window size: 10 tokens before and 10 tokens after to get wider context
    min_count=5, # only consider tokens with at least n occurrences in the corpus
    negative=15, # negative subsampling: bigger than default to sample negative examples more
    min_n=2, # min character n-gram
    max_n=5 # max character n-gram
)
ft_model.build_vocab(tok_text) # tok_text is our tokenized input text - a list of lists relating to docs and tokens respectivley

ft_model.train(
    tok_text,
    epochs=6,
    total_examples=ft_model.corpus_count, 
    total_words=ft_model.corpus_total_words)

ft_model.save('_fasttext.model') #save
ft_model = FastText.load('_fasttext.model') #load

In [ ]:
ft_model = FastText.load('_fasttext.model')

In [ ]:
with plt.xkcd():
  pd.DataFrame(ft_model.wv.most_similar("london", topn=10, restrict_vocab=5000),columns=['Word','Score']).plot.barh(x='Word',figsize=(6,6),color=(0.3,0.7,0.7))

In [ ]:
with plt.xkcd():
  pd.DataFrame(ft_model.wv.most_similar("england", topn=10, restrict_vocab=10000),columns=['Word','Score']).plot.barh(x='Word',figsize=(6,6),color=(0.3,0.7,0.7))

In [ ]:
bm25 = BM25Okapi(tok_text)
weighted_doc_vects = []

for i, doc in tqdm(enumerate(tok_text)):
    doc_vector = []
    for word in doc:
        try:
            vector = ft_model[word]
            weight = (bm25.idf[word] * ((bm25.k1 + 1.0)*bm25.doc_freqs[i][word])) / (bm25.k1 * (1.0 - bm25.b + bm25.b *(bm25.doc_len[i]/bm25.avgdl))+bm25.doc_freqs[i][word])
            weighted_vector = vector * weight
            doc_vector.append(weighted_vector)
        except KeyError:
            continue
    doc_vector_mean = np.mean(doc_vector,axis=0)
    weighted_doc_vects.append(doc_vector_mean)

0it [00:00, ?it/s]<ipython-input-14-74daef8bf42d>:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vector = ft_model[word]
220it [00:00, 531.26it/s]


In [ ]:
pickle.dump( weighted_doc_vects, open( "weighted_doc_vects.p", "wb" ) )

In [ ]:
with open( "weighted_doc_vects.p", "rb" ) as f:
  weighted_doc_vects = pickle.load(f)
# create a random matrix to index
data = np.vstack(weighted_doc_vects)

# initialize a new index, using a HNSW index on Cosine Similarity - can take a couple of mins
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(data)
index.createIndex({'post': 2}, print_progress=True)

In [ ]:
# querying the index:
input = 'Emotional'.lower().split()


query = [ft_model[vec] for vec in input]
query = np.mean(query,axis=0)

t0 = time.time()
ids, distances = index.knnQuery(query, k=10)
t1 = time.time()
print(f'Searched {df.shape[0]} records in {round(t1-t0,4) } seconds \n')
for i,j in zip(ids,distances):
  print(round(j,2))
  print(df.text.values[i])

Searched 220 records in 0.0003 seconds 

0.01
Tell Us About Your Care Partnerships The Care Quality Commission (CQC) is the independent regulatory of health and adult social care in England. CQC is responsible for monitoring and inspecting these services to ensure they are providing care that is safe, effective, caring and responsive to people's needs and well-led listening to acting on people's experiences of care is ... Publication date 27 February 2023 Closing 29 March 2023, 12pm Contract value £250,000 Contract location NE1 4PA
0.02
427 - Emotional and Mental Health Service for Children in Care and Care Leavers Wigan Council's requirements of an Emotional and Mental Health (EMH) Service for our Children in Care and our young people who have left our care; including children with neurodevelopmental / SEND / LD. The service is to be a flexible and a responsive service for this cohort of children and young people who have in the vast majority of cases, ... Publication date 27 February

<ipython-input-25-65377f99f21a>:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  query = [ft_model[vec] for vec in input]
